# Find minimum Temperature by Location from Weather Data

### RDD Approach

In [7]:
val lines = sc.textFile("/Users/pulkit/data/1800.csv")

In [27]:
val wSMin = lines.map(x => { val y = x.split(",")
                                (y(0).toString,y(2).toString,y(3).toInt) }
                                ).
                                filter( x => x._2 =="TMIN").
                                map(x => (x._1,x._3.toFloat))

In [33]:
import scala.math.min

val wS = wSMin.reduceByKey((x,y) => min(x,y)).sortBy(x => x._2)

In [34]:
val result = wS.take(5).foreach(println)

(ITE00100554,-148.0)
(EZE00100082,-135.0)


### DataFrame Approach

In [37]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.col
val spark2 = spark
import spark2.implicits._



val wdf = spark2.read.
                    format("csv").
                    option("header","false").
                    load("/Users/pulkit/data/1800.csv")



In [38]:
wdf.show(4)

+-----------+--------+----+----+----+----+---+----+
|        _c0|     _c1| _c2| _c3| _c4| _c5|_c6| _c7|
+-----------+--------+----+----+----+----+---+----+
|ITE00100554|18000101|TMAX| -75|null|null|  E|null|
|ITE00100554|18000101|TMIN|-148|null|null|  E|null|
|GM000010962|18000101|PRCP|   0|null|null|  E|null|
|EZE00100082|18000101|TMAX| -86|null|null|  E|null|
+-----------+--------+----+----+----+----+---+----+
only showing top 4 rows



In [58]:
val wMaxTemp = wdf.select($"_c0".as("Station"),$"_c2".as("TempType"),$"_c3".as("Temperature")).
                                where($"TempType"==="TMIN").
                                groupBy($"Station").agg(max($"Temperature")).
                                withColumnRenamed("max(Temperature)", "MaxTemp")

In [59]:
wMaxTemp.show()

+-----------+-------+
|    Station|MaxTemp|
+-----------+-------+
|ITE00100554|     99|
|EZE00100082|     99|
+-----------+-------+

